# 🎯 Fabric Spark Advisor — Interactive Notebook

**AI-powered Spark optimization agent for Microsoft Fabric workloads**

---

## Features

- ✅ **Interactive Chat UI** with ipywidgets
- ✅ **Back-and-forth conversations** with context retention
- ✅ **Professional card layout** for recommendations
- ✅ **Feedback buttons** for continuous learning
- ✅ **Real-time Kusto queries** for live telemetry data
- ✅ **3-tier source priority** (Kusto → RAG → LLM)

---

## 1️⃣ Setup & Imports

In [ ]:
# Install required dependencies (run once)
# %pip install ipywidgets pandas azure-kusto-data azure-kusto-ingest python-dotenv

import sys
import os
from pathlib import Path

# Add parent directory to path to import agent modules
project_root = Path().absolute().parent
sys.path.insert(0, str(project_root))

import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from datetime import datetime
import uuid

# Import Spark Advisor components
from agent.orchestrator import SparkAdvisorOrchestrator
from mcp_server.kusto_client import KustoClient
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

print("✅ Imports successful!")

: 

## 2️⃣ Initialize Spark Advisor

In [ ]:
# Initialize components
kusto_client = KustoClient.from_env()
orchestrator = SparkAdvisorOrchestrator.from_env()
session_id = str(uuid.uuid4())

# Session state for conversation context
conversation_history = []
last_query_context = {
    "query_text": None,
    "response_text": None,
    "intent": None,
    "application_id": None
}

print(f"✅ Spark Advisor initialized! Session ID: {session_id[:8]}...")

## 3️⃣ Branding & Styling

In [ ]:
e# FSA Premium Branding CSS
branding_css = """
<style>
    @import url('https://fonts.googleapis.com/css2?family=Inter:wght@400;500;600;700&display=swap');
    
    * {
        font-family: 'Inter', 'Segoe UI', -apple-system, BlinkMacSystemFont, sans-serif;
    }
    
    .fsa-header {
        background: linear-gradient(135deg, #0099CC 0%, #00D4FF 50%, #B388FF 100%);
        padding: 32px 24px;
        border-radius: 16px;
        margin-bottom: 24px;
        text-align: center;
        color: white;
        box-shadow: 0 8px 32px rgba(0, 153, 204, 0.25), 0 2px 8px rgba(0, 0, 0, 0.1);
        position: relative;
        overflow: hidden;
    }
    .fsa-header::before {
        content: '';
        position: absolute;
        top: 0;
        left: 0;
        right: 0;
        bottom: 0;
        background: radial-gradient(circle at 30% 50%, rgba(255, 255, 255, 0.1) 0%, transparent 60%);
        pointer-events: none;
    }
    .fsa-logo {
        font-size: 36px;
        font-weight: 700;
        text-shadow: 0 2px 12px rgba(0, 0, 0, 0.2);
        letter-spacing: -0.5px;
        position: relative;
        z-index: 1;
    }
    .fsa-tagline {
        font-size: 14px;
        font-weight: 500;
        opacity: 0.95;
        margin-top: 12px;
        letter-spacing: 0.2px;
        position: relative;
        z-index: 1;
    }
    
    .recommendation-card {
        background: linear-gradient(145deg, #1a1f2e 0%, #151a26 100%);
        border-left: 4px solid;
        border-radius: 12px;
        padding: 20px;
        margin: 12px 0;
        box-shadow: 0 4px 16px rgba(0, 0, 0, 0.2), 0 1px 4px rgba(0, 0, 0, 0.1);
        transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1);
        position: relative;
        overflow: hidden;
    }
    .recommendation-card::before {
        content: '';
        position: absolute;
        top: 0;
        left: 0;
        width: 4px;
        height: 100%;
        opacity: 0;
        transition: opacity 0.3s ease;
    }
    .recommendation-card:hover {
        transform: translateY(-2px);
        box-shadow: 0 8px 24px rgba(0, 0, 0, 0.3), 0 2px 8px rgba(0, 0, 0, 0.15);
    }
    .recommendation-card:hover::before {
        opacity: 0.5;
    }
    
    .card-critical { border-color: #FF5252; }
    .card-critical::before { background: #FF5252; }
    .card-high { border-color: #FFB300; }
    .card-high::before { background: #FFB300; }
    .card-medium { border-color: #29B6F6; }
    .card-medium::before { background: #29B6F6; }
    .card-low { border-color: #3FB950; }
    .card-low::before { background: #3FB950; }
    
    .card-header {
        font-weight: 600;
        font-size: 16px;
        margin-bottom: 12px;
        color: #00D4FF;
        letter-spacing: -0.2px;
    }
    .card-body {
        color: #e1e4e8;
        line-height: 1.7;
        font-size: 14px;
    }
    
    .source-badge {
        display: inline-block;
        padding: 5px 12px;
        border-radius: 16px;
        font-size: 10px;
        font-weight: 600;
        margin-right: 8px;
        text-transform: uppercase;
        letter-spacing: 0.5px;
        box-shadow: 0 2px 8px rgba(0, 0, 0, 0.15);
    }
    .badge-kusto { 
        background: linear-gradient(135deg, #3FB950 0%, #2da544 100%);
        color: white;
    }
    .badge-rag { 
        background: linear-gradient(135deg, #29B6F6 0%, #1e88e5 100%);
        color: white;
    }
    .badge-llm { 
        background: linear-gradient(135deg, #B388FF 0%, #9c64ff 100%);
        color: white;
    }
    
    .ai-warning {
        background: linear-gradient(145deg, #2d1b3d 0%, #251733 100%);
        border: 2px dashed rgba(179, 136, 255, 0.5);
        border-radius: 12px;
        padding: 20px;
        margin: 12px 0;
        color: #e1e4e8;
        box-shadow: 0 4px 16px rgba(179, 136, 255, 0.1);
    }
</style>
"""

display(HTML(branding_css))

# Display header
header_html = """
<div class="fsa-header">
    <div class="fsa-logo">🎯 Fabric Spark Advisor</div>
    <div class="fsa-tagline">AI-powered Spark optimization for Microsoft Fabric workloads</div>
</div>
"""

display(HTML(header_html))
print("✅ Branding loaded!")

## 4️⃣ Helper Functions

In [ ]:
def format_recommendation_card(title, content, severity="MEDIUM", source="KUSTO"):
    """Format a single recommendation as a styled card"""
    severity_class_map = {
        "CRITICAL": "card-critical",
        "HIGH": "card-high",
        "MEDIUM": "card-medium",
        "LOW": "card-low"
    }
    
    badge_class_map = {
        "KUSTO": "badge-kusto",
        "RAG": "badge-rag",
        "LLM": "badge-llm"
    }
    
    severity_class = severity_class_map.get(severity.upper(), "card-medium")
    badge_class = badge_class_map.get(source.upper(), "badge-kusto")
    
    return f"""
    <div class="recommendation-card {severity_class}">
        <div class="card-header">
            <span class="source-badge {badge_class}">{source.upper()}</span>
            {title}
        </div>
        <div class="card-body">{content}</div>
    </div>
    """

def format_ai_warning(content):
    """Format LLM-generated content with warning box"""
    return f"""
    <div class="ai-warning">
        <strong>⚠️ AI GENERATED — NOT FROM YOUR DATA</strong><br>
        <em>Source: LLM training knowledge | Validate before applying to production</em>
        <hr style="border-color: #B388FF; margin: 10px 0;">
        {content}
    </div>
    """

def save_feedback(feedback_type, comment=""):
    """Save user feedback to Kusto"""
    if last_query_context["query_text"]:
        try:
            success = kusto_client.insert_feedback(
                session_id=session_id,
                application_id=last_query_context.get("application_id", "N/A"),
                query_text=last_query_context["query_text"],
                query_intent=last_query_context.get("intent", "general_chat"),
                actual_result_generated=last_query_context.get("response_text", "")[:5000],
                feedback_type=feedback_type,
                feedback_comment=comment,
                recommendation_count=0,
                kusto_recommendation_count=0,
                rag_recommendation_count=0,
                llm_recommendation_count=0
            )
            return success
        except Exception as e:
            print(f"❌ Error saving feedback: {e}")
            return False
    return False

print("✅ Helper functions loaded!")

## 5️⃣ Interactive Chat UI

In [ ]:
# Premium widget styling
widget_style = """
<style>
    .widget-textarea textarea {
        background: linear-gradient(145deg, #1a1f2e 0%, #151a26 100%) !important;
        border: 2px solid rgba(0, 212, 255, 0.2) !important;
        border-radius: 12px !important;
        padding: 16px !important;
        color: #e1e4e8 !important;
        font-family: 'Inter', 'Segoe UI', sans-serif !important;
        font-size: 14px !important;
        line-height: 1.6 !important;
        transition: all 0.3s ease !important;
        box-shadow: 0 2px 8px rgba(0, 0, 0, 0.1) !important;
    }
    .widget-textarea textarea:focus {
        border-color: #00D4FF !important;
        box-shadow: 0 0 0 3px rgba(0, 212, 255, 0.1), 0 4px 16px rgba(0, 0, 0, 0.15) !important;
        outline: none !important;
    }
    .widget-textarea textarea::placeholder {
        color: #5A7A8A !important;
        opacity: 0.7 !important;
    }
    .widget-button button {
        border-radius: 10px !important;
        padding: 10px 24px !important;
        font-weight: 600 !important;
        font-size: 14px !important;
        transition: all 0.3s cubic-bezier(0.4, 0, 0.2, 1) !important;
        box-shadow: 0 2px 8px rgba(0, 0, 0, 0.15) !important;
        border: none !important;
    }
    .widget-button button:hover {
        transform: translateY(-2px) !important;
        box-shadow: 0 4px 16px rgba(0, 0, 0, 0.25) !important;
    }
    .widget-button button:active {
        transform: translateY(0) !important;
    }
    .output_wrapper {
        border-radius: 16px !important;
        overflow: hidden !important;
    }
</style>
"""
display(HTML(widget_style))

# Create premium widgets
chat_output = widgets.Output(
    layout={
        'border': '2px solid rgba(0, 212, 255, 0.2)',
        'height': '550px',
        'overflow_y': 'auto',
        'padding': '16px',
        'border_radius': '16px',
        'background': 'linear-gradient(145deg, #0d1318 0%, #0a0f14 100%)'
    }
)

query_input = widgets.Textarea(
    placeholder='Ask me about Spark applications... Try: "analyze application_YOUR_APP_ID" or "show top 5 slowest apps"',
    layout={'width': '100%', 'height': '90px'}
)

send_button = widgets.Button(
    description='🚀 Send',
    button_style='info',
    layout={'width': '130px', 'height': '42px'},
    tooltip='Send your query'
)

clear_button = widgets.Button(
    description='🗑️ Clear',
    button_style='warning',
    layout={'width': '130px', 'height': '42px'},
    tooltip='Clear conversation history'
)

# Premium example queries card
examples_html = HTML("""
<div style="margin: 16px 0 24px 0; padding: 20px 24px; 
            background: linear-gradient(145deg, #1a1f2e 0%, #151a26 100%);
            border-radius: 12px; border-left: 4px solid #00D4FF;
            box-shadow: 0 4px 16px rgba(0, 0, 0, 0.15);">
    <div style="display: flex; align-items: center; margin-bottom: 16px;">
        <span style="font-size: 24px; margin-right: 12px;">💡</span>
        <strong style="color: #00D4FF; font-size: 16px; font-weight: 600; letter-spacing: -0.2px;">Example Queries</strong>
    </div>
    <div style="display: grid; gap: 8px;">
        <div style="padding: 10px 14px; background: rgba(0, 212, 255, 0.05); 
                    border-radius: 8px; border-left: 3px solid rgba(0, 212, 255, 0.3);">
            <code style="color: #00D4FF; font-size: 13px; font-family: 'Consolas', 'Monaco', monospace;">analyze application_YOUR_APP_ID</code>
        </div>
        <div style="padding: 10px 14px; background: rgba(0, 212, 255, 0.05); 
                    border-radius: 8px; border-left: 3px solid rgba(0, 212, 255, 0.3);">
            <code style="color: #00D4FF; font-size: 13px; font-family: 'Consolas', 'Monaco', monospace;">show top 5 slowest applications</code>
        </div>
        <div style="padding: 10px 14px; background: rgba(0, 212, 255, 0.05); 
                    border-radius: 8px; border-left: 3px solid rgba(0, 212, 255, 0.3);">
            <code style="color: #00D4FF; font-size: 13px; font-family: 'Consolas', 'Monaco', monospace;">find streaming jobs</code>
        </div>
        <div style="padding: 10px 14px; background: rgba(0, 212, 255, 0.05); 
                    border-radius: 8px; border-left: 3px solid rgba(0, 212, 255, 0.3);">
            <code style="color: #00D4FF; font-size: 13px; font-family: 'Consolas', 'Monaco', monospace;">what is VOrder?</code>
        </div>
        <div style="padding: 10px 14px; background: rgba(0, 212, 255, 0.05); 
                    border-radius: 8px; border-left: 3px solid rgba(0, 212, 255, 0.3);">
            <code style="color: #00D4FF; font-size: 13px; font-family: 'Consolas', 'Monaco', monospace;">show bad practice apps</code>
        </div>
    </div>
</div>
""")

def create_feedback_widget(response_id):
    """Create feedback buttons and comment box for a specific response"""
    feedback_helpful = widgets.Button(description='✅ Helpful', button_style='success', layout={'width': '130px'})
    feedback_not_helpful = widgets.Button(description='❌ Not Helpful', button_style='danger', layout={'width': '130px'})
    feedback_partial = widgets.Button(description='⚠️ Partially Helpful', button_style='warning', layout={'width': '150px'})
    feedback_comment = widgets.Textarea(
        placeholder='Optional: Add your detailed feedback...',
        layout={'width': '100%', 'height': '60px', 'margin': '10px 0 0 0'}
    )
    
    # Container for acknowledgment message
    ack_output = widgets.Output()
    
    def handle_feedback_click(feedback_type):
        def on_click(b):
            comment = feedback_comment.value.strip()
            success = save_feedback(feedback_type, comment)
            
            # Disable buttons after submission
            feedback_helpful.disabled = True
            feedback_not_helpful.disabled = True
            feedback_partial.disabled = True
            feedback_comment.disabled = True
            
            # Show acknowledgment
            with ack_output:
                clear_output()
                if success:
                    ack_html = f"""
                    <div style="margin: 10px 0; padding: 12px; background: linear-gradient(135deg, #1a4d2e 0%, #2d5f3f 100%); 
                                border-left: 4px solid #3FB950; border-radius: 6px;">
                        <strong style="color: #3FB950;">✅ Feedback Submitted Successfully!</strong><br>
                        <span style="color: #e1e4e8; font-size: 13px;">
                            Rated as: <strong>{feedback_type}</strong><br>
                            {f'Your comment: "{comment}"<br>' if comment else ''}
                            <em>Thank you! Your feedback helps improve future recommendations.</em>
                        </span>
                    </div>
                    """
                else:
                    ack_html = """
                    <div style="margin: 10px 0; padding: 12px; background: #2d1b1b; border-left: 4px solid #FFB300; border-radius: 6px;">
                        <strong style="color: #FFB300;">⚠️ Feedback Not Saved</strong><br>
                        <span style="color: #e1e4e8; font-size: 13px;">Could not save feedback. Please try again.</span>
                    </div>
                    """
                display(HTML(ack_html))
        
        return on_click
    
    feedback_helpful.on_click(handle_feedback_click("HELPFUL"))
    feedback_not_helpful.on_click(handle_feedback_click("NOT_HELPFUL"))
    feedback_partial.on_click(handle_feedback_click("PARTIAL"))
    
    # Feedback section with branding
    feedback_header = widgets.HTML("""
        <div style="margin: 15px 0 10px 0; padding: 10px; background: linear-gradient(90deg, #0099CC 0%, #00D4FF 100%); 
                    border-radius: 6px 6px 0 0; border-bottom: 2px solid #00D4FF;">
            <strong style="color: white; font-size: 14px;">💬 Was this helpful?</strong>
            <span style="color: rgba(255,255,255,0.9); font-size: 12px; margin-left: 10px;">
                Your feedback improves future recommendations
            </span>
        </div>
    """)
    
    button_row = widgets.HBox(
        [feedback_helpful, feedback_not_helpful, feedback_partial],
        layout={'margin': '10px 0'}
    )
    
    feedback_box = widgets.VBox([
        feedback_header,
        button_row,
        feedback_comment,
        ack_output
    ], layout={'margin': '0 0 20px 0'})
    
    return feedback_box

def send_query(b):
    """Handle send button click with premium animations"""
    query = query_input.value.strip()
    if not query:
        return
    
    with chat_output:
        # Display user question with premium styling
        timestamp = datetime.now().strftime("%H:%M:%S")
        user_html = f"""
        <div style="margin: 16px 0; padding: 16px 20px; 
                    background: linear-gradient(145deg, #0d1318 0%, #0a0f14 100%);
                    border-left: 4px solid #00D4FF; border-radius: 12px;
                    box-shadow: 0 2px 8px rgba(0, 0, 0, 0.1);
                    animation: slideIn 0.3s ease-out;">
            <div style="display: flex; align-items: center; margin-bottom: 8px;">
                <span style="font-size: 20px; margin-right: 10px;">👤</span>
                <strong style="color: #00D4FF; font-weight: 600;">You</strong>
                <span style="color: #5A7A8A; font-size: 11px; margin-left: auto; font-weight: 500;">{timestamp}</span>
            </div>
            <div style="color: #e1e4e8; font-size: 14px; line-height: 1.6; padding-left: 30px;">{query}</div>
        </div>
        """
        display(HTML(user_html))
        
        # Show premium loading indicator
        loading_html = """
        <div style="margin: 16px 0; padding: 16px 20px;
                    background: linear-gradient(145deg, rgba(0, 212, 255, 0.05) 0%, rgba(0, 212, 255, 0.02) 100%);
                    border-left: 4px solid rgba(0, 212, 255, 0.5);
                    border-radius: 12px;
                    animation: pulse 1.5s ease-in-out infinite;">
            <div style="display: flex; align-items: center;">
                <div style="font-size: 20px; margin-right: 12px; animation: spin 2s linear infinite;">⚙️</div>
                <span style="color: #00D4FF; font-weight: 500;">Analyzing your query...</span>
            </div>
        </div>
        <style>
            @keyframes pulse {
                0%, 100% { opacity: 1; }
                50% { opacity: 0.6; }
            }
            @keyframes spin {
                from { transform: rotate(0deg); }
                to { transform: rotate(360deg); }
            }
            @keyframes slideIn {
                from { opacity: 0; transform: translateY(-10px); }
                to { opacity: 1; transform: translateY(0); }
            }
        </style>
        """
        display(HTML(loading_html))
        
        try:
            # Get response from orchestrator
            import asyncio
            
            # Run async chat method
            loop = asyncio.get_event_loop()
            response = loop.run_until_complete(orchestrator.chat(query, session_id=session_id))
            
            # Update context
            last_query_context["query_text"] = query
            last_query_context["response_text"] = response
            conversation_history.append({"query": query, "response": response, "timestamp": timestamp})
            
            # Clear loading message
            clear_output(wait=True)
            
            # Display user question again
            display(HTML(user_html))
            
            # Display response with premium styling
            response_html = f"""
            <div style="margin: 16px 0; padding: 20px 24px;
                        background: linear-gradient(145deg, rgba(63, 185, 80, 0.08) 0%, rgba(63, 185, 80, 0.02) 100%);
                        border-left: 4px solid #3FB950; border-radius: 12px;
                        box-shadow: 0 4px 16px rgba(0, 0, 0, 0.1);
                        animation: slideIn 0.3s ease-out;">
                <div style="display: flex; align-items: center; margin-bottom: 12px;">
                    <span style="font-size: 24px; margin-right: 12px;">🎯</span>
                    <strong style="color: #3FB950; font-weight: 600; font-size: 15px;">Fabric Spark Advisor</strong>
                    <span style="color: #5A7A8A; font-size: 11px; margin-left: auto; font-weight: 500;">{timestamp}</span>
                </div>
                <div style="color: #e1e4e8; font-size: 14px; line-height: 1.7; padding-left: 36px;">
                    {response.replace(chr(10), '<br>')}
                </div>
            </div>
            """
            display(HTML(response_html))
            
            # Add feedback widget after each response
            response_id = str(uuid.uuid4())[:8]
            feedback_widget = create_feedback_widget(response_id)
            display(feedback_widget)
            
        except Exception as e:
            clear_output(wait=True)
            display(HTML(user_html))
            error_html = f"""
            <div style="margin: 16px 0; padding: 20px 24px;
                        background: linear-gradient(145deg, rgba(255, 82, 82, 0.1) 0%, rgba(255, 82, 82, 0.02) 100%);
                        border-left: 4px solid #FF5252; border-radius: 12px;
                        box-shadow: 0 4px 16px rgba(255, 82, 82, 0.1);">
                <div style="display: flex; align-items: center; margin-bottom: 8px;">
                    <span style="font-size: 20px; margin-right: 10px;">❌</span>
                    <strong style="color: #FF5252; font-weight: 600;">Error Processing Query</strong>
                </div>
                <div style="color: #e1e4e8; font-size: 13px; padding-left: 30px; line-height: 1.6;">{str(e)}</div>
            </div>
            """
            display(HTML(error_html))
    
    # Clear input
    query_input.value = ''

def clear_chat(b):
    """Clear chat history"""
    with chat_output:
        clear_output()
        display(HTML('<div style="color: #8b949e; text-align: center; padding: 20px;">Chat cleared. Start a new conversation!</div>'))
    conversation_history.clear()

def handle_feedback(feedback_type):
    """Handle feedback button clicks (legacy - for global feedback section if needed)"""
    def on_click(b):
        comment = feedback_comment.value.strip()
        success = save_feedback(feedback_type, comment)
        
        with chat_output:
            if success:
                msg_html = f'<div style="color: #3FB950; margin: 10px 0;">✅ Thank you! Your {feedback_type} feedback has been saved.</div>'
            else:
                msg_html = '<div style="color: #FFB300; margin: 10px 0;">⚠️ Could not save feedback (no recent query).</div>'
            display(HTML(msg_html))
        
        feedback_comment.value = ''
    
    return on_click

# Connect event handlers
send_button.on_click(send_query)
clear_button.on_click(clear_chat)

# Premium Layout with better spacing
input_box = widgets.HBox([query_input], layout={'margin': '0 0 12px 0'})
button_box = widgets.HBox(
    [send_button, clear_button], 
    layout={'margin': '0 0 20px 0', 'gap': '12px'}
)

# Display UI
display(examples_html)
display(input_box)
display(button_box)
display(chat_output)

# Premium welcome message
with chat_output:
    welcome_html = """
    <div style="text-align: center; padding: 60px 20px; animation: fadeIn 0.6s ease-out;">
        <div style="font-size: 48px; margin-bottom: 20px;">🎯</div>
        <div style="color: #00D4FF; font-size: 22px; font-weight: 600; margin-bottom: 12px; letter-spacing: -0.5px;">
            Ready to optimize your Spark workloads!
        </div>
        <div style="color: #8b949e; font-size: 14px; line-height: 1.6; max-width: 400px; margin: 0 auto;">
            Type your query in the text area above and click <strong style="color: #00D4FF;">🚀 Send</strong> to get started.<br>
            I can analyze applications, find bottlenecks, and provide optimization recommendations.
        </div>
    </div>
    <style>
        @keyframes fadeIn {
            from { opacity: 0; transform: translateY(10px); }
            to { opacity: 1; transform: translateY(0); }
        }
    </style>
    """
    display(HTML(welcome_html))

print("✅ Interactive chat UI ready!")

## 6️⃣ Advanced: Direct API Access

In [ ]:
# For users who want to call methods directly instead of using the chat UI

# Example: Get top 5 slowest applications
query = 'sparklens_metrics | where metric == "Total Executor Time (sec)" | top 5 by value desc'
results = kusto_client.query_to_dict_list(query)

print("📊 Top 5 Slowest Applications:")
for row in results:
    app_id = row.get('application_id', 'N/A')
    value = row.get('value', 0)
    print(f"  • {app_id}: {value:.1f} seconds")

In [ ]:
# Example: Analyze a specific application
# CHANGE THIS to your application ID
app_id = "application_1771441543262_0001"  # Replace with your app ID

# Get recommendations directly from Kusto
spark_recs_data = kusto_client.get_sparklens_recommendations(app_id)
fabric_recs_data = kusto_client.get_fabric_recommendations(app_id)

print(f"🔍 Analysis for {app_id}:")
print(f"\n📌 Spark Advisor Recommendations: {len(spark_recs_data) if spark_recs_data else 0}")
print(f"📌 Fabric Recommendations: {len(fabric_recs_data) if fabric_recs_data else 0}")

if spark_recs_data:
    print(f"\n📄 Spark Advisor Recommendation Content:")
    for i, rec in enumerate(spark_recs_data[:1], 1):  # Show first one
        print(f"\n{rec.get('recommendation', 'No recommendation text')[:500]}...")
else:
    print("\n⚠️ No Spark Advisor data found in Kusto for this app")

if fabric_recs_data:
    print(f"\n📄 Fabric Recommendation Content:")
    for i, rec in enumerate(fabric_recs_data[:1], 1):  # Show first one
        print(f"\n{rec.get('recommendation', 'No recommendation text')[:500]}...")
else:
    print("\n⚠️ No Fabric data found in Kusto for this app")

# Diagnostic: List all available apps if current app not found
if not spark_recs_data and not fabric_recs_data:
    print("\n🔍 Checking what apps are available in Kusto...\n")
    try:
        available_apps = kusto_client.query_to_dict_list(
            "sparklens_recommedations | distinct app_id | take 10"
        )
        if available_apps:
            print(f"📋 Sample apps available in Kusto (first 10):")
            for app in available_apps:
                print(f"   • {app.get('app_id', 'N/A')}")
        else:
            print("❌ No apps found in recommendations table")
    except Exception as e:
        print(f"❌ Error querying Kusto: {e}")

## 7️⃣ Session Statistics

In [ ]:
# Display session statistics
print(f"📊 Session Statistics")
print(f"  Session ID: {session_id}")
print(f"  Queries Asked: {len(conversation_history)}")
print(f"  Started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

if conversation_history:
    print(f"\n📜 Recent Queries:")
    for i, conv in enumerate(conversation_history[-5:], 1):
        print(f"  {i}. [{conv['timestamp']}] {conv['query'][:60]}...")

## 8️⃣ Diagnostics & Troubleshooting

In [ ]:
# Diagnostic tool to check if a specific app exists in Kusto
def check_app_in_kusto(app_id):
    """Check if an application has data in Kusto tables"""
    print(f"🔍 Diagnostic Check for: {app_id}")
    print("=" * 60)
    
    # Check Spark Advisor recommendations table
    try:
        query1 = f"sparklens_recommedations | where app_id == '{app_id}' | count"
        result1 = kusto_client.query_to_dict_list(query1)
        count1 = result1[0]['Count'] if result1 else 0
        
        if count1 > 0:
            print(f"✅ Spark Advisor recommendations: {count1} record(s) found")
            # Get sample
            sample = kusto_client.query_to_dict_list(
                f"sparklens_recommedations | where app_id == '{app_id}' | take 1"
            )
            if sample and sample[0].get('recommendation'):
                rec_preview = sample[0]['recommendation'][:200]
                print(f"   Preview: {rec_preview}...")
        else:
            print(f"❌ Spark Advisor recommendations: No records found")
    except Exception as e:
        print(f"❌ Error querying Spark Advisor recommendations: {e}")
    
    # Check Fabric recommendations table
    try:
        query2 = f"fabric_recommedations | where app_id == '{app_id}' | count"
        result2 = kusto_client.query_to_dict_list(query2)
        count2 = result2[0]['Count'] if result2 else 0
        
        if count2 > 0:
            print(f"✅ Fabric recommendations: {count2} record(s) found")
            # Get sample
            sample = kusto_client.query_to_dict_list(
                f"fabric_recommedations | where app_id == '{app_id}' | take 1"
            )
            if sample and sample[0].get('recommendation'):
                rec_preview = sample[0]['recommendation'][:200]
                print(f"   Preview: {rec_preview}...")
        else:
            print(f"❌ Fabric recommendations: No records found")
    except Exception as e:
        print(f"❌ Error querying Fabric recommendations: {e}")
    
    # Check metrics table
    try:
        query3 = f"sparklens_metrics | where application_id == '{app_id}' | count"
        result3 = kusto_client.query_to_dict_list(query3)
        count3 = result3[0]['Count'] if result3 else 0
        print(f"{'✅' if count3 > 0 else '❌'} Performance metrics: {count3} record(s)")
    except Exception as e:
        print(f"❌ Error querying performance metrics: {e}")
    
    print("=" * 60)

# Usage: Replace with your actual application ID
# check_app_in_kusto("application_XXXXXXXXX_XXXX")

print("✅ Diagnostic function loaded! Use: check_app_in_kusto('your_app_id')")

In [ ]:
# List all available applications in Kusto
def list_available_apps(limit=20):
    """List applications available in Kusto for testing"""
    print(f"📋 Available Applications in Kusto (showing first {limit}):\n")
    
    try:
        # Get apps from Spark Advisor recommendations
        query = f"sparklens_recommedations | distinct app_id | take {limit}"
        apps = kusto_client.query_to_dict_list(query)
        
        if apps:
            print(f"✅ Found {len(apps)} apps with Spark Advisor recommendations:")
            for i, app in enumerate(apps, 1):
                app_id = app.get('app_id', 'N/A')
                print(f"   {i}. {app_id}")
        else:
            print("❌ No apps found in Spark Advisor recommendations table")
            
        print(f"\n{'='*60}\n")
        
        # Get apps from Fabric recommendations
        query2 = f"fabric_recommedations | distinct app_id | take {limit}"
        fabric_apps = kusto_client.query_to_dict_list(query2)
        
        if fabric_apps:
            print(f"✅ Found {len(fabric_apps)} apps with Fabric recommendations:")
            for i, app in enumerate(fabric_apps, 1):
                app_id = app.get('app_id', 'N/A')
                print(f"   {i}. {app_id}")
        else:
            print("❌ No apps found in Fabric recommendations table")
            
    except Exception as e:
        print(f"❌ Error querying Kusto: {e}")

# Usage: Run this to see what app IDs are available
# list_available_apps(10)

print("✅ List function loaded! Use: list_available_apps(20)")

## 🧪 Quick Test: Auto-Analyze First Available App

In [ ]:
# 🚀 QUICK TEST: Automatically pick and analyze an app from Kusto
# This is useful for testing the end-to-end workflow

print("🧪 Quick Test: Auto-Analyzing First Available App")
print("=" * 70)

try:
    # Step 1: Get first available app
    print("\n📋 Step 1: Fetching available apps from Kusto...")
    query = "sparklens_recommedations | distinct app_id | take 1"
    apps = kusto_client.query_to_dict_list(query)
    
    if not apps:
        print("❌ No apps found in Kusto. Please check:")
        print("   1. Your Kusto connection (.env file)")
        print("   2. Data has been ingested into sparklens_recommedations table")
        print("\nRun this to see all tables: kusto_client.query_to_dict_list('.show tables')")
    else:
        test_app_id = apps[0].get('app_id', 'N/A')
        print(f"✅ Found app: {test_app_id}")
        
        # Step 2: Check what data exists for this app
        print(f"\n🔍 Step 2: Checking data availability...")
        check_app_in_kusto(test_app_id)
        
        # Step 3: Get recommendations directly
        print(f"\n📄 Step 3: Retrieving recommendations from Kusto...")
        spark_recs = kusto_client.get_sparklens_recommendations(test_app_id)
        fabric_recs = kusto_client.get_fabric_recommendations(test_app_id)
        
        print(f"   Spark Advisor: {len(spark_recs) if spark_recs else 0} recommendation(s)")
        print(f"   Fabric:        {len(fabric_recs) if fabric_recs else 0} recommendation(s)")
        
        # Step 4: Show sample content
        if spark_recs:
            print(f"\n📝 Sample Spark Advisor Recommendation:")
            print("-" * 70)
            rec_text = spark_recs[0].get('recommendation', 'No text')
            print(rec_text[:500] + "..." if len(rec_text) > 500 else rec_text)
            print("-" * 70)
        
        if fabric_recs:
            print(f"\n📝 Sample Fabric Recommendation:")
            print("-" * 70)
            rec_text = fabric_recs[0].get('recommendation', 'No text')
            print(rec_text[:500] + "..." if len(rec_text) > 500 else rec_text)
            print("-" * 70)
        
        # Step 5: Success message
        print(f"\n✅ Quick Test Complete!")
        print(f"\n💡 Next Steps:")
        print(f"   • Try in Chat UI: analyze {test_app_id}")
        print(f"   • List more apps: list_available_apps(10)")
        print(f"   • Check specific app: check_app_in_kusto('your_app_id')")
        
except Exception as e:
    print(f"❌ Test failed with error: {e}")
    print("\n🔧 Troubleshooting:")
    print("   1. Check .env file has correct Kusto credentials")
    print("   2. Verify network connectivity to Kusto cluster")
    print("   3. Confirm tables exist: sparklens_recommedations, fabric_recommedations")

print("\n" + "=" * 70)

---

## 📚 Documentation

- **System Architecture**: See `ARCHITECTURE.md`
- **MCP Tools Reference**: See `components/TOOL_REFERENCE.md`
- **Hallucination Prevention**: See `components/HALLUCINATION_PREVENTION.md`
- **Feedback Learning**: See `mcp_server/feedback_learning_strategy.md`

---

## 🔒 Security Notes

- Ensure `.env` file contains valid Kusto credentials
- Never commit credentials to version control
- Use Managed Identity or Service Principal for production

---

**Built with ❤️ by the Fabric Spark Advisor Team**